In [1]:
from cesnet_datazoo.datasets import CESNET_QUIC22
from cesnet_datazoo.config import DatasetConfig, AppSelection, ValidationApproach

dataset = CESNET_QUIC22("~/datasets/CESNET-QUIC22/", size="XS")

common_params = {
    "dataset": dataset,
    "apps_selection": AppSelection.ALL_KNOWN,
    "train_period_name": "W-2022-44",
    "val_approach": ValidationApproach.SPLIT_FROM_TRAIN,
    "train_val_split_fraction": 0.2,
    "use_packet_histograms": True,
}
dataset_config = DatasetConfig(**common_params)
dataset.set_dataset_config_and_initialize(dataset_config)
train_dataframe = dataset.get_train_df(flatten_ppi=True)
val_dataframe = dataset.get_val_df(flatten_ppi=True)
test_dataframe = dataset.get_test_df(flatten_ppi=True)

Loading data from dataloader


100%|██████████| 8162/8162 [00:09<00:00, 863.54it/s] 


Loading data from dataloader


100%|██████████| 192/192 [00:04<00:00, 45.18it/s]


Loading data from dataloader


100%|██████████| 1247/1247 [00:09<00:00, 135.33it/s]


In [3]:
from river import ensemble
from river import tree

model = ensemble.BaggingClassifier(
    model=tree.HoeffdingTreeClassifier(),
    n_models=10
)

In [5]:
from river import datasets
from river import metrics
from river.stream import iter_array

X_train = train_dataframe.drop(columns="APP").to_numpy()[:10000]
y_train = train_dataframe["APP"].to_numpy()[:10000]

dataset = iter_array(X_train, y_train)
metric = metrics.Accuracy()

for x, y in dataset:
    y_pred = model.predict_one(x)
    metric.update(y, y_pred)
    model.learn_one(x, y)

print(f'Accuracy: {metric}')

Accuracy: Accuracy: 9.24%
